<a href="https://colab.research.google.com/github/iammiori/DataAnalysisAndPractice/blob/master/yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ijson

     |████████████████████████████████| 61kB 2.3MB/s 


## 1. ijson 설치이유
- json 모듈을 이용해, yelp의 review data를 불러오면, 메모리에러가 뜸.
- 메모리에러는 노트북상 램 문제인듯 싶었으나, 램이 16기가인 데스크탑도 마찬가지 였음.
- 구글링 결과,  json의 한계점이며, ijson 모듈을 찾음.

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## 2. Colab 사용과 마운트
- 노트북이나, 데스크탑의 jupyter로 돌릴경우, cpu를 활용하기 때문에 대용량 파일을 읽고 쓰기에는 속도와 시간 메모리 문제가 많이 생김.
- colab 의 경우, GPU를 끌어다 쓸 수 있기 때문에 코드를 빨리 돌릴수 있을거라 생각해 colab환경에서 데이터를 불러옴
- colab에서 데이터를 불러오기위해, 구글드라이브에 접근을 하여야 하므로 마운트는 꼭 해줘야함.(위에 코드가 마운트를 해주는 코드)

In [0]:
import io
import ijson

user = []
store =[]
rating =[]

with open("/content/gdrive/My Drive/review.json",encoding="UTF-8") as json_file:
    cursor=0
    for line_number,line in enumerate(json_file):
        cnt=0
        #print("{},{}".format(line_number+1,cursor))
        line_as_file = io.StringIO(line)
        
        json_parser = ijson.parse(line_as_file)
        for prefix,type,value in json_parser:
            cnt+=1
            if cnt in [5,7,9]:
                if cnt==5:
                    user.append(value)
                elif cnt==7:
                    store.append(value)
                else:
                    rating.append(float(value))
                
        cursor += len(line)
print("finish")

finish


## 3. json 을 parsing 해서 읽어오면서 필요한 정보  list에 저장
```
{"review_id":"Q1sbwvVQXV2734tPgoKj4Q","user_id":"hG7b0MtEbXx5QzbzE6C_VA","business_id":"ujmEBvifdJM6h6RLv4wQIg","stars":1.0,"useful":6,"funny":1,"cool":0,"text":"Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.","date":"2013-05-07 04:34:36"}
```

- review.json의 형태는 이런 식인데, 위의 코드는 한 줄씩 읽어오는 코드고, 또  cursor 변수가 '{' 를  null "review_id" 는 2 review_id의 값은 3 , "user_id" 는 4, user_id값은 5 이런식으로 하나씩 늘려가면서 받아온다

- 따라서 우리가 필요한 정보는 user,business,stars 값이기 때문에 cursor가 5,7,9 에 있을 때의  json 값을 갖고오면 된다.

- 가져온 값은  list에 저장한다.


In [0]:
len(user)

6685900

In [0]:
len(store)

6685900

In [0]:
len(rating)

6685900

가져온 결과, 6685900 약 6백만개의 data를 가져왔다

In [0]:
import pandas as pd
df = pd.DataFrame(columns=[["user","store","rating"]])

## 4. DataFrame 생성
- user, store, rating column 으로 dataframe 을 만들어준다.

In [0]:
df

,user,store,rating


In [0]:
df["user"]=user

In [0]:
df["store"]=store

In [0]:
df["rating"]=rating

In [0]:
df.head()

,user,store,rating
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0
1,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0
2,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0
3,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0
4,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0


In [0]:
df.to_csv("/content/gdrive/My Drive/review.csv",index=False)

In [0]:
import pandas as pd
df1 = pd.read_csv("/content/gdrive/My Drive/review.csv")

In [3]:
df1.shape

(6685900, 3)

그 결과, 모든 결과가 csv에 들어갔으나, 큰 용량 때문에, 엑셀에서 로드가 되지 않는다. 
또한 6백만개의 데이터는 많기 때문에 sampling을 하기로 하였다.

In [4]:
df1.head()

,user,store,rating
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0
1,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0
2,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0
3,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0
4,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0


## 5. Data Sampling


In [22]:
#전체 약 6백7만개 정도 되니 데이터 축소를 위해 10%로 샘플링
df_sample = df1.sample(frac=0.016,random_state=777).reset_index()
df_sample.head()

,index,user,store,rating
0,4574077,RAqriks-b0qAyNjbhX-Vfw,cKmhPywftFx1iPaBjvvr7Q,5.0
1,4929990,79XVZ6YyBuytf6GkkaDPLA,Jur1iiejUFw_YCkfDfy1gQ,4.0
2,1732551,2SLzz7CHvmVMhKuISyqKtQ,LNGBEEelQx4zbfWnlc66cw,4.0
3,2310044,jzC4DBdaXCiRtFUvv2bKLQ,f4x1YBxkLrZg652xt2KR5g,1.0
4,1323756,8KHlpXRfn_CSnQRy_ghOJQ,qHA-e0jhtXj5BC5XawUSJw,5.0


In [23]:
df_sample = df_sample.drop(["index"],axis=1)
df_sample.head()

,user,store,rating
0,RAqriks-b0qAyNjbhX-Vfw,cKmhPywftFx1iPaBjvvr7Q,5.0
1,79XVZ6YyBuytf6GkkaDPLA,Jur1iiejUFw_YCkfDfy1gQ,4.0
2,2SLzz7CHvmVMhKuISyqKtQ,LNGBEEelQx4zbfWnlc66cw,4.0
3,jzC4DBdaXCiRtFUvv2bKLQ,f4x1YBxkLrZg652xt2KR5g,1.0
4,8KHlpXRfn_CSnQRy_ghOJQ,qHA-e0jhtXj5BC5XawUSJw,5.0


In [24]:
df_sample.shape

(106974, 3)

In [0]:
df_sample.to_csv("/content/gdrive/My Drive/review_sample.csv",index=False)

sampling 한 결과 최종 파일 저장 완료

- sampling frac 을 0.1로 했을 때, unstack 을 하는 과정에서 램용량 오류발생.

## 6. User-Item matrix 생성


In [0]:
df_sampling = pd.read_csv("/content/gdrive/My Drive/review_sample.csv")

In [27]:
df_sampling.head()

,user,store,rating
0,RAqriks-b0qAyNjbhX-Vfw,cKmhPywftFx1iPaBjvvr7Q,5.0
1,79XVZ6YyBuytf6GkkaDPLA,Jur1iiejUFw_YCkfDfy1gQ,4.0
2,2SLzz7CHvmVMhKuISyqKtQ,LNGBEEelQx4zbfWnlc66cw,4.0
3,jzC4DBdaXCiRtFUvv2bKLQ,f4x1YBxkLrZg652xt2KR5g,1.0
4,8KHlpXRfn_CSnQRy_ghOJQ,qHA-e0jhtXj5BC5XawUSJw,5.0


In [0]:
ui_matrix = df_sampling.set_index(["user","store"],append=True).unstack()
ui_matrix.head()